In [1]:
!pip install -r ../requirements.txt

In [2]:
import os  # os를 가져와 파일 시스템 접근, 환경 변수 읽을 수 있음
from openai import OpenAI  # OpenAI의 api 사용 가능
import chromadb  # chromadb 라이브러리 쓸 수 있게 해줌
from chromadb.config import (
    Settings,
)  # Settings 클래스는 DB의 구성 옵션을 설정하는데 사용
from dotenv import load_dotenv  # 환경 변수를 로드하기 위함

1. 환경 변수 Load해서 api_key 가져오고 OpenAI 클라이언트(객체) 초기화

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

2. DB 초기화 함수 (매 실행 시 DB 폴더를 삭제 후 새로 생성)

In [4]:
def init_db(db_path="./chroma_db"): # 현재 디렉토리 내의 chroma_db폴더 설정
    dbclient = chromadb.PersistentClient(path=db_path) # 지정 경로로 향하는 dbClient생성
    collection = dbclient.create_collection(name="rag_collection", get_or_create=True)
    return dbclient, collection

3. 텍스트 로딩 함수

In [5]:
def load_text_files(folder_path):
    docs = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if file_path.endswith(".txt"):
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()
                docs.append((filename, text))
    return docs            

4. 주어진 text를 임베딩 벡터로 변환하는 함수

In [6]:
def get_embedding(text, model="text-embedding-3-large"):
    response = client.embeddings.create(input=[text], model=model)
    embedding = response.data[0].embedding
    return embedding


5. 원천 데이터 청크 단위로 나누고 overlap 사이즈 조절하는 함수

In [7]:
def chunk_text(text, chunk_size=400, chunk_overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start+chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - chunk_overlap
        
        if start<0:
            start = 0

        if start >= len(text):
            break

    return chunks

6. 문서로드 -> 청크 나누고 -> 임베딩 생성 후 DB 삽입

In [8]:
if __name__ == "__main__":
    # db 초기화, 경로 지정
    dbclient, collection = init_db("./chroma_db")

    # load_text_files 함수로 처리할 문서 데이터 불러오기기
    folder_path = "./source_data"
    docs = load_text_files(folder_path)

    doc_id = 0

    for filename, text in docs:
        chunks = chunk_text(text, chunk_size=400, chunk_overlap=50)
        for idx, chunk in enumerate(chunks):
            doc_id += 1
            embedding = get_embedding(chunk)

            collection.add(
                documents=[chunk],
                embeddings=[embedding],
                metadatas=[{"filename": filename, "chunk_index": idx}],
                ids=[str(doc_id)]
            )

    print("모든 문서 벡터DB에 저장 완료")

모든 문서 벡터DB에 저장 완료
